# Googl Calendar Api 使ってみる
## 処理フローは：
* GoogleカレンダのJobカレンダから全イベントを取得
* Outlookから指定期間内のイベントでZoomとWebExと思われるものを抽出
* EntryIdをキーに突合してUpdate/Insertを行う
* 期間外やキャンセルされたと思われるイベントの削除

In [46]:
#from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import re
import win32com.client

In [48]:
import json
# load setting file(json)
with open("calendar.json", "r", encoding="utf-8") as f:
    dic = json.load(f)

# "id":"xxxxx@group.calendar.google.com"
calendar_id = dic["id"]

# "tools": {"zoom": "zoom.us","webex": "webex.com", "teams": "teams.microsoft.com"}
tools = dic["tools"]

if "omit" in dic:
    omit=dic["omit"]
else:
    omit=[]

if "weeks" in dic:
    weeks_ahead=dic["weeks"]
else:
    weeks_ahead=3

for k in tools:
    print(k,tools[k])
print("omit",omit)
print("weeks ahead",weeks_ahead)

zoom zoom.us
webex webex.com
teams teams.microsoft.com
omit ['DBSV-CPU']
weeks ahead 4


In [49]:
# 期間：自
st = datetime.date.today() - datetime.timedelta(days=1)
# 期間：至
ed = datetime.date.today() + datetime.timedelta(weeks=weeks_ahead)

# Googleカレンダのやつ
SCOPES = ["https://www.googleapis.com/auth/calendar"]
cred_json = "credentials.json"

# トークンの読み込み
creds = None
if os.path.exists("token.pickle"):
    with open("token.pickle", "rb") as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(cred_json, SCOPES)
        creds = flow.run_local_server(port=0)
    with open("token.pickle", "wb") as token:
        pickle.dump(creds, token)

# Googleカレンダサービスの取得
service = build("calendar", "v3", credentials=creds)

# Googleカレンダのイベント取得
print("Getting all events")
event_result = (
    service.events()
    .list(
        calendarId=calendar_id,
        #                                     timeMin=st.isoformat()+'T00:00:00Z',
        #                                     timeMax=ed.isoformat()+'T00:00:00Z',
        singleEvents=True,
        orderBy="startTime",
    )
    .execute()
)
events = event_result.get("items", [])

# 表示してみよう
if not events:
    print("No upcoming events found.")
for i, event in enumerate(events):
    start = event["start"].get("dateTime", event["start"].get("date"))
    start = datetime.datetime.strptime(start[:-6], "%Y-%m-%dT%H:%M:%S")
    print("{:02}".format(i), start, event["summary"])

Getting all events
00 2021-04-27 09:30:00 zoom:案件MTG
01 2021-04-27 13:30:00 teams:PMO部課長会議
02 2021-04-27 15:00:00 teams:地域案件Ｔ支援Ｇ定例(毎火15時)
03 2021-04-27 16:00:00 teams:地域G内レビュー_審査報告書（鹿児島庶務事務案件）
04 2021-04-27 18:00:00 zoom:重案審（G事業統括部長主査）
05 2021-04-28 09:00:00 zoom:重案審（G事業統括部長主査）
06 2021-04-28 09:30:00 teams:コメント票の確認_竹中工務店試験審査
07 2021-04-28 14:00:00 teams:【日程変更】グループ会社案件担当定例MTG（地域定例）
08 2021-04-28 15:30:00 zoom:重要案件審議会（G事業統括部長主査）
09 2021-04-28 16:00:00 teams:Ｄ九州（鹿児島庶務事務案件）設計審査のご説明
10 2021-04-28 16:30:00 webex: Webex ミーティング招待状: 【公社】分野連携ミーティング
11 2021-05-07 10:00:00 teams:Ｄ東北（ＹＢ）設計審査・事前打合せ
12 2021-05-10 10:00:00 teams:5月高リスク案件報告事前確認
13 2021-05-10 15:00:00 teams:Ｄ東北（ＹＢ）Ｑ／Ａ表ＲＶ①
14 2021-05-10 17:00:00 teams:FW: PJ体制構築ガイド等の改訂案説明(公社/法ソリ）
15 2021-05-11 13:30:00 teams:PMO部課長会議
16 2021-05-11 15:00:00 teams:地域案件Ｔ支援Ｇ定例(毎火15時)
17 2021-05-11 16:00:00 teams:Ｄ東北（ＹＢ）Ｑ／Ａ表ＲＶ②
18 2021-05-13 13:30:00 teams:グループ会社案件担当定例MTG（地域定例）
19 2021-05-13 16:00:00 teams:Ｄ東北（ＹＢ）設計審査・報告書ＲＶ
20 2021-05-14 13:00:00 zoom:D関西・竹中P

In [50]:
# online meetings
tools = {"zoom": "zoom.us", "webex": "webex.com", "teams": "teams.microsoft.com"}

# OutlookのエントリIDを持っている、GoogleイベントID取得
def getEventId(events, entryId):
    for i, ev in enumerate(events):
        if entryId in ev["description"]:
            return i  # , ev["id"], ev["updated"]
    return -1  # , None, None


# OutlookのエントリIDと開始時間が一致する場合のみ同一判定
def getEventIndex(events, item):
    for i, ev in enumerate(events):
        if item.EntryId in ev["description"]:
            start = ev["start"].get("dateTime", ev["start"].get("date"))
            # timzoneつけないとそもそも比較できないので付加→astimezone()
            # なぜかわからんがOutlookはUTCタイム、astimezone後はJSTになっちゃうのでUTCに変換する必要あり
            start = (
                datetime.datetime.strptime(start[:-6], "%Y-%m-%dT%H:%M:%S")
                .astimezone()
                .replace(tzinfo=datetime.timezone.utc)
            )
            td = start - item.Start
            # print(item.Start,start,td)
            if td == datetime.timedelta(0):
                return i
    return -1


# 対象判定
def isOnline(item):
    for k, v in tools.items():
        if v in item.Body:
            for sub in omit:
                if sub in item.Subject:
                    return False, None
            return True, k
    return False, None


# Outlookの予定アクセス用のAPI
app = win32com.client.Dispatch("Outlook.Application")
root = app.Session.DefaultStore.GetRootFolder()
ns = app.GetNamespace("MAPI")
cal = ns.GetDefaultFolder(9)

# 期間指定用のフィルタ文字列
filterStr = '[Start]>="{0}" and [Start]<"{1}"'.format(
    st.strftime("%Y/%m/%d"), ed.strftime("%Y/%m/%d")
)
print("filter", filterStr)

# ソートして定期的な予定を分解してからフィルタかける
appointments = cal.Items
appointments.sort("[Start]")
appointments.IncludeRecurrences = True
restricted = appointments.Restrict(filterStr)
print("from", st, "to", ed)
counter = 0

# Outlook側の予定アイテムにアクセス
for item in restricted:
    # BodyにZoom.usもしくはwebex.comって記載があるやつだけを対象にする
    ret, prefix = isOnline(item)
    countStr = "{:02}".format(counter)
    if ret:
        # 件名の先頭にprefixをつける
        summary = prefix + ":" + item.subject

        # Body部の組み立て。パスワードまでをコピー
        bd = item.body.splitlines()
        # desc = item.subject + "\n"
        desc = ""
        for line in bd:
            desc += line + "\n"
            if "パスワード" in line:
                break
        desc += "EntryId={}".format(item.EntryId)
        # Googleカレンダ用のイベントJSon組み立て
        event = {
            "summary": summary,
            "description": desc,
            "start": {
                "dateTime": item.Start.strftime("%Y-%m-%dT%H:%M:%S"),
                "timeZone": "Japan",
            },
            "end": {
                "dateTime": item.End.strftime("%Y-%m-%dT%H:%M:%S"),
                "timeZone": "Japan",
            },
        }

        # 検索済みのイベントと突合
        index = getEventIndex(events, item)

        if index < 0:
            # 突合しなかったので、新規追加
            print(countStr, "insert", item.Start.isoformat(), item.subject)
            service.events().insert(
                calendarId=calendar_id,
                body=event).execute()
        else:
            # POPして突合したイベントリストから消しておく
            ev = events.pop(index)
            updated = datetime.datetime.fromisoformat(
                ev["updated"].replace("Z", "+00:00")
            )
            modified = item.LastModificationTime + datetime.timedelta(hours=-9)
            # print(modified.isoformat(),updated)
            if updated < modified:
                # 突合したので、内容に関わらず更新しとく
                print(countStr, "update", item.Start.isoformat(), item.subject)
                service.events().update(
                    calendarId=calendar_id, 
                    eventId=ev["id"], 
                    body=event
                ).execute()
            else:
                print(countStr, "no upd", item.Start.isoformat(), item.subject)
            start = ev["start"].get("dateTime", ev["start"].get("date"))
            start = datetime.datetime.strptime(start[:-6], "%Y-%m-%dT%H:%M:%S")
        counter += 1

# 期間外もしくは削除済みイベントを削除
for event in events:
    print("delete", event["start"]["dateTime"], event["summary"])
    service.events().delete(
        calendarId=calendar_id, 
        eventId=event["id"]).execute()
print("done.")

filter [Start]>="2021/04/27" and [Start]<"2021/05/26"
from 2021-04-27 to 2021-05-26
00 no upd 2021-04-27T09:30:00+00:00 案件MTG
01 update 2021-04-27T13:30:00+00:00 PMO部課長会議
02 update 2021-04-27T15:00:00+00:00 地域案件Ｔ支援Ｇ定例(毎火15時)
03 update 2021-04-27T16:00:00+00:00 地域G内レビュー_審査報告書（鹿児島庶務事務案件）
04 no upd 2021-04-27T18:00:00+00:00 重案審（G事業統括部長主査）
05 update 2021-04-28T09:00:00+00:00 重案審（G事業統括部長主査）
06 update 2021-04-28T09:30:00+00:00 コメント票の確認_竹中工務店試験審査
07 no upd 2021-04-28T14:00:00+00:00 【日程変更】グループ会社案件担当定例MTG（地域定例）
08 no upd 2021-04-28T15:30:00+00:00 重要案件審議会（G事業統括部長主査）
09 no upd 2021-04-28T16:00:00+00:00 Ｄ九州（鹿児島庶務事務案件）設計審査のご説明
10 no upd 2021-04-28T16:30:00+00:00  Webex ミーティング招待状: 【公社】分野連携ミーティング
11 no upd 2021-05-07T10:00:00+00:00 Ｄ東北（ＹＢ）設計審査・事前打合せ
12 no upd 2021-05-10T10:00:00+00:00 5月高リスク案件報告事前確認
13 no upd 2021-05-10T15:00:00+00:00 Ｄ東北（ＹＢ）Ｑ／Ａ表ＲＶ①
14 no upd 2021-05-10T17:00:00+00:00 FW: PJ体制構築ガイド等の改訂案説明(公社/法ソリ）
15 update 2021-05-11T13:30:00+00:00 PMO部課長会議
16 update 2021-05-11T15:00:00+00:00 地域案件Ｔ支援